# Exercise 3 - Oasis Keys Service

This exercise shows how to use the inbuilt keys service code to map exposure data to the model specific keys (area peril and vulnerability)

In [51]:
# standard libraries
import json

import geopandas as gpd
import pandas as pd

from shapely.geometry import Polygon



In [4]:
# oed exposure data
df_loc = pd.read_csv('test_data/location.csv')

df_loc

,PortNumber,AccNumber,LocNumber,CountryCode,Latitude,Longitude,StreetAddress,PostalCode,City,AreaCode,...,RoofDeck,RoofGeometry,RoofPitch,RoofAnchorage,RoofDeckAttachment,RoofCoverAttachment,RoofYearBuilt,RoofFrame,WindowProtection,BuildingExteriorOpening
0,ARA,11111,3,US,0.0,0.0,NaN,85335,NaN,AZ,...,0,0,0,0,0,0,0,0,0,0
1,ARA,11111,4,US,0.0,0.0,NaN,85301,NaN,AZ,...,0,0,0,0,0,0,0,0,0,0
2,ARA,11111,5,US,0.0,0.0,NaN,85301,NaN,AZ,...,0,0,0,0,0,0,0,0,0,0
3,ARA,11111,6,US,0.0,0.0,NaN,85210,NaN,AZ,...,0,0,0,0,0,0,0,0,0,0
4,ARA,11111,7,US,0.0,0.0,NaN,85022,NaN,AZ,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,ARA,11111,449,US,0.0,0.0,NaN,91367,NaN,CA,...,0,0,0,0,0,0,0,0,0,0
443,ARA,11111,450,US,0.0,0.0,NaN,92805,NaN,CA,...,0,0,0,0,0,0,0,0,0,0
444,ARA,11111,451,US,0.0,0.0,NaN,33024,NaN,FL,...,0,0,0,0,0,0,0,0,0,0
445,ARA,11111,452,US,0.0,0.0,NaN,33024,NaN,FL,...,0,0,0,0,0,0,0,0,0,0


In [10]:
! oasislmf model generate-keys --help

usage: oasislmf model generate-keys [-h] [-V] [-C CONFIG]
                                    [-x OED_LOCATION_CSV] [-k KEYS_DATA_CSV]
                                    [-e KEYS_ERRORS_CSV] [-f {oasis,json}]
                                    [-g LOOKUP_CONFIG_JSON]
                                    [-d LOOKUP_DATA_DIR]
                                    [-l LOOKUP_MODULE_PATH]
                                    [-L LOOKUP_COMPLEX_CONFIG_JSON]
                                    [--lookup-num-processes LOOKUP_NUM_PROCESSES]
                                    [--lookup-num-chunks LOOKUP_NUM_CHUNKS]
                                    [-v MODEL_VERSION_CSV] [-D USER_DATA_DIR]

Generates keys from a model lookup, and write Oasis keys and keys error files.

optional arguments:
  -h, --help            show this help message and exit
  -V, --verbose         Use verbose logging.
  -C CONFIG, --config CONFIG
                        MDK config. JSON file
  -x OED_LOCATION_CSV, --oed-loc

In [69]:
lookup_config = {
    "model": {
        "supplier_id": "OasisLMF",
        "model_id": "OasisHurdat",
        "model_version": "0.0.1"
    },
    "builtin_lookup_type": "new_lookup",
    "keys_data_path": "keys_data",
    "step_definition": {
        "area_peril":{
            "type": "rtree",
            "columns": ["latitude", "longitude"],
            "parameters": {
                "file_path": "keys_data/areaperil_dict.parquet",
                "file_type": "parquet",
                "id_columns": ["area_peril_id"], 
                "nearest_neighbor_min_distance": -1
            }
        },
        "split_loc_perils_covered":{
            "type": "split_loc_perils_covered" ,
            "columns": ["locperilscovered"],
            "parameters": {
                "model_perils_covered": ["WTC"]
            }
        },
        "create_coverage_type": {
            "type": "simple_pivot",
            "parameters": {
                "pivots": [{"new_cols": {"coverage_type": 1}}]
            }
        },
        "vulnerability": {
            "type": "merge",
            "columns": ["peril_id", "coverage_type", "occupancycode"],
            "parameters": {"file_path": "keys_data/vulnerability_dict.csv",
                           "id_columns": ["vulnerability_id"]
                          }
        }
    },
    "strategy": ["split_loc_perils_covered", "area_peril", "create_coverage_type", "vulnerability"]
}

lookup_config


{'model': {'supplier_id': 'OasisLMF',
  'model_id': 'OasisHurdat',
  'model_version': '0.0.1'},
 'builtin_lookup_type': 'new_lookup',
 'keys_data_path': 'keys_data',
 'step_definition': {'area_peril': {'type': 'rtree',
   'columns': ['latitude', 'longitude'],
   'parameters': {'file_path': 'keys_data/areaperil_dict.parquet',
    'file_type': 'parquet',
    'id_columns': ['area_peril_id'],
    'nearest_neighbor_min_distance': -1}},
  'split_loc_perils_covered': {'type': 'split_loc_perils_covered',
   'columns': ['locperilscovered'],
   'parameters': {'model_perils_covered': ['WTC']}},
  'create_coverage_type': {'type': 'simple_pivot',
   'parameters': {'pivots': [{'new_cols': {'coverage_type': 1}}]}},
  'vulnerability': {'type': 'merge',
   'columns': ['peril_id', 'coverage_type', 'occupancycode'],
   'parameters': {'file_path': 'keys_data/vulnerability_dict.csv',
    'id_columns': ['vulnerability_id']}}},
 'strategy': ['split_loc_perils_covered',
  'area_peril',
  'create_coverage_type

In [70]:
# write config file
with open('lookup_config.json','w') as lc:
    json.dump(lookup_config, lc)
    
! ls -ltr

total 456
-rw-rw-r--  1 ben ben     14 Aug 30 21:52 README.md
-rw-rw-r--  1 ben ben   1538 Aug 30 21:52 LICENSE
drwxrwxr-x  2 ben ben   4096 Aug 30 21:57 meta-data
drwxrwxr-x  2 ben ben   4096 Aug 31 09:47 source_data
drwxrwxr-x  2 ben ben   4096 Sep  1 23:26 model_data
-rw-rw-r--  1 ben ben  15601 Sep  2 08:06 Exercise_2_Vulnerability.ipynb
drwxrwxr-x  2 ben ben   4096 Sep  2 08:09 test_data
-rw-rw-r--  1 ben ben 372718 Sep  2 09:34 Exercise_1_Hazard.ipynb
drwxrwxr-x  2 ben ben   4096 Sep  2 09:45 keys_data
drwxrwxr-x 12 ben ben   4096 Sep  2 09:50 runs
-rw-rw-r--  1 ben ben  38597 Sep  2 09:51 Untitled.ipynb
-rw-rw-r--  1 ben ben    965 Sep  2 09:51 lookup_config.json


In [71]:
# generate areaperil grid 
df_grid = pd.read_csv('keys_data/areaperil_dict.csv')

df_grid['lon1'] = df_grid['longitude']-0.05
df_grid['lon2'] = df_grid['longitude']-0.05
df_grid['lon3'] = df_grid['longitude']+0.05
df_grid['lon4'] = df_grid['longitude']+0.05

df_grid['lat1'] = df_grid['latitude']-0.05
df_grid['lat2'] = df_grid['latitude']+0.05
df_grid['lat3'] = df_grid['latitude']-0.05
df_grid['lat4'] = df_grid['latitude']+0.05

df_grid['peril_id'] = 'WTC'

polygon_point_order = [1, 2, 4, 3]

gdf_peril_area = gpd.GeoDataFrame(df_grid)
gdf_peril_area["geometry"] = gdf_peril_area.apply(
    lambda row: Polygon([(row[f"lon{i}"], row[f"lat{i}"]) for i in polygon_point_order]), axis=1)


gfd_file_df = gdf_peril_area[['peril_id','area_peril_id','geometry']]

gfd_file_df.to_parquet('keys_data/areaperil_dict.parquet')

gfd_file_df


<ipython-input-71-59f9de9a711e>:25: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gfd_file_df.to_parquet('keys_data/areaperil_dict.parquet')


,peril_id,area_peril_id,geometry
0,WTC,1,"POLYGON ((-81.35000 24.65000, -81.35000 24.750..."
1,WTC,2,"POLYGON ((-81.35000 25.85000, -81.35000 25.950..."
2,WTC,3,"POLYGON ((-81.35000 25.95000, -81.35000 26.050..."
3,WTC,4,"POLYGON ((-81.35000 26.05000, -81.35000 26.150..."
4,WTC,5,"POLYGON ((-81.35000 26.15000, -81.35000 26.250..."
...,...,...,...
37704,WTC,37705,"POLYGON ((-67.25000 44.85000, -67.25000 44.950..."
37705,WTC,37706,"POLYGON ((-67.25000 44.95000, -67.25000 45.050..."
37706,WTC,37707,"POLYGON ((-67.15000 44.75000, -67.15000 44.850..."
37707,WTC,37708,"POLYGON ((-67.15000 44.95000, -67.15000 45.050..."


In [72]:
with open('keys_data/vulnerability_dict.csv','w') as v:
    v.write('PERIL_ID,COVERAGE_TYPE,OCCUPANCYCODE,VULNERABILITY_ID\n')
    v.write('WTC,1,1101,1')
    
! cat keys_data/vulnerability_dict.csv 

PERIL_ID,COVERAGE_TYPE,OCCUPANCYCODE,VULNERABILITY_ID
WTC,1,1101,1

In [77]:
! oasislmf model generate-keys -x 'test_data/location.csv' -g 'lookup_config.json' -d 'keys_data'


Stating oasislmf command - GenerateKeys
RUNNING: oasislmf.manager.interface
RUNNING: oasislmf.lookup.factory.generate_key_files
COMPLETED: oasislmf.lookup.factory.generate_key_files in 3.18s

Keys successful: /home/ben/oasis/git/Workshop2021/runs/keys-20210902095359/keys.csv generated with 11 items
Keys errors: /home/ben/oasis/git/Workshop2021/runs/keys-20210902095359/keys-errors.csv generated with 434 items
COMPLETED: oasislmf.manager.interface in 3.18s


In [74]:
! head <<<'copy in "Keys successful" path from above message'>>>

LocID,PerilID,CoverageTypeID,AreaPerilID,VulnerabilityID
444,WTC,1,36625,1
69,WTC,1,1526,1
70,WTC,1,1526,1
73,WTC,1,25887,1
74,WTC,1,1419,1
75,WTC,1,1418,1
76,WTC,1,25738,1
117,WTC,1,36987,1
198,WTC,1,9930,1
